<a href="https://colab.research.google.com/github/t-nakatani/signate_stu22/blob/main/signate_stu_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

from google.colab import drive
drive.mount('/content/drive')
ROOT = "/content/drive/MyDrive/Colab Notebooks/signate/"

#学習用データと評価用データの読み込み
train = pd.read_csv(os.path.join(ROOT, "train_cleaned.csv"))
test = pd.read_csv(os.path.join(ROOT, "test_cleaned.csv"))

jobdic = dict(zip([1, 2, 3, 4], ['DataScientist', 'ML Engineer', 'Software Engineer', 'Consultant']))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


https://qiita.com/yamaru/items/63a342c844cff056a549

In [10]:
train

,id,description,jobflag
0,0,Develop cutting edge applications that perform...,3
1,1,Designs develops high quality scalable efficie...,3
2,2,Functions point person Network Strategy work R...,4
3,3,Work technical design development release depl...,3
4,4,Quantify resources required task project relat...,4
...,...,...,...
1511,1511,Support detailed reporting statistical analyse...,1
1512,1512,Collaborate with teams support technical roadm...,2
1513,1513,Work with executives other business leaders id...,1
1514,1514,Leading design ideation sessions ensure buildi...,3


In [9]:
train.description[0]

'Develop cutting edge applications that perform superbly across platforms Work highly collaborative environment with cross functional teams projects ranging from weeks months length Maintain high standard quality creatively strategically problem solve throughout product delivery process able effectively communicate your work with both technical technical peersBe excited about technologies techniquesBuild solid front architectures that integrate easily with other systems technologiesWorking closely with other disciplines Back Design Have superior attention detail strong ability work required including cross browser cross platform displays performance'

In [5]:
!pip install -q transformers==3

     |████████████████████████████████| 754 kB 15.1 MB/s 
     |████████████████████████████████| 880 kB 68.3 MB/s 
     |████████████████████████████████| 1.2 MB 60.9 MB/s 
     |████████████████████████████████| 3.0 MB 23.0 MB/s 


In [14]:
import numpy as np
import transformers
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizerFast, RobertaModel
from torch import optim
from torch import cuda
import time
from matplotlib import pyplot as plt
from tqdm import tqdm

In [7]:
# Datasetの定義
class CreateDataset(Dataset):
  def __init__(self, X, y, tokenizer, max_len):
    self.X = X
    self.y = y
    self.tokenizer = tokenizer
    self.max_len = max_len

  def __len__(self):  # len(Dataset)で返す値を指定
    return len(self.y)

  def __getitem__(self, index):  # Dataset[index]で返す値を指定
    text = self.X[index]
    inputs = self.tokenizer.encode_plus(
      text,
      add_special_tokens=True,
      max_length=self.max_len,
      pad_to_max_length=True,
      truncation=True
    )
    ids = inputs['input_ids']
    mask = inputs['attention_mask']

    return {
      'ids': torch.LongTensor(ids),
      'mask': torch.LongTensor(mask),
      'labels': torch.Tensor(self.y[index])
    }

In [ ]:
from sklearn.model_selection import train_test_split

y = pd.get_dummies(train.jobflag).values
# y_test = pd.get_dummies(test.jobflag).values

X_train, X_val, y_train, y_val = train_test_split(train['description'].values, y, test_size=0.2,random_state=109) 

# Datasetの作成
max_len = 400
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base')
dataset_train = CreateDataset(X_train, y_train, tokenizer, max_len)
dataset_val = CreateDataset(X_val, y_val, tokenizer, max_len)
dataset_test = CreateDataset(test['description'], [-1]*len(test['description']), tokenizer, max_len)

for var in dataset_train[0]:
  print(f'{var}: {dataset_train[0][var]}')

In [15]:
# BERT分類モデルの定義
class BERTClass(torch.nn.Module):
  def __init__(self, drop_rate, otuput_size):
    super().__init__()
    self.bert = RobertaModel.from_pretrained('roberta-base')
    self.drop = torch.nn.Dropout(drop_rate)
    self.fc = torch.nn.Linear(768, otuput_size)  # BERTの出力に合わせて768次元を指定

  def forward(self, ids, mask):
    _, out = self.bert(ids, attention_mask=mask)
    out = self.fc(self.drop(out))
    return out

In [19]:
def calculate_loss_and_accuracy(model, criterion, loader, device):
  """ 損失・正解率を計算"""
  model.eval()
  loss = 0.0
  total = 0
  correct = 0
  with torch.no_grad():
    for data in tqdm(loader):
      # デバイスの指定
      ids = data['ids'].to(device)
      mask = data['mask'].to(device)
      labels = data['labels'].to(device)

      # 順伝播
      outputs = model(ids, mask)

      # 損失計算
      loss += criterion(outputs, labels).item()

      # 正解率計算
      pred = torch.argmax(outputs, dim=-1).cpu().numpy() # バッチサイズの長さの予測ラベル配列
      labels = torch.argmax(labels, dim=-1).cpu().numpy()  # バッチサイズの長さの正解ラベル配列
      total += len(labels)
      correct += (pred == labels).sum().item()

  return loss / len(loader), correct / total

In [20]:
# パラメータの設定
DROP_RATE = 0.1
OUTPUT_SIZE = 4
BATCH_SIZE = 16
NUM_EPOCHS = 10
LEARNING_RATE = 1e-6

# モデルの定義
model = BERTClass(DROP_RATE, OUTPUT_SIZE)

# 損失関数の定義
criterion = torch.nn.CrossEntropyLoss()

# オプティマイザの定義
optimizer = torch.optim.AdamW(params=model.parameters(), lr=LEARNING_RATE)

# デバイスの指定
device = 'cuda' if cuda.is_available() else 'cpu'

# モデルの学習
# log = train_model(dataset_train, dataset_val, BATCH_SIZE, model, criterion, optimizer, NUM_EPOCHS, device=device)

In [21]:
best_acc = 0
model.to(device)

# dataloaderの作成
dataloader_train = DataLoader(dataset_train, batch_size=BATCH_SIZE, shuffle=True)
dataloader_valid = DataLoader(dataset_val, batch_size=BATCH_SIZE, shuffle=False)

# 学習
log_train = []
log_valid = []
for epoch in range(NUM_EPOCHS):
  # 開始時刻の記録
  s_time = time.time()


  model.train()
  for data in tqdm(dataloader_train):
    ids = data['ids'].to(device)
    mask = data['mask'].to(device)
    labels = data['labels'].to(device)

    optimizer.zero_grad()

    outputs = model(ids, mask)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

  # 損失と正解率の算出
  loss_train, acc_train = calculate_loss_and_accuracy(model, criterion, dataloader_train, device)
  loss_valid, acc_valid = calculate_loss_and_accuracy(model, criterion, dataloader_valid, device)
  log_train.append([loss_train, acc_train])
  log_valid.append([loss_valid, acc_valid])
  if best_acc < acc_valid:
    best_acc = acc_valid
    
    weight_name = "v3_best.pth"
    torch.save(model, os.path.join(ROOT, weight_name))
    print(f'best-model saved (epoch: {epoch + 1})')

  # 終了時刻の記録
  e_time = time.time()

  # ログを出力
  print(f'epoch: {epoch + 1}, loss_train: {loss_train:.4f}, accuracy_train: {acc_train:.4f}, loss_valid: {loss_valid:.4f}, accuracy_valid: {acc_valid:.4f}, {(e_time - s_time):.4f}sec') 
  
weight_name = "v3_last.pth"
torch.save(model, os.path.join(ROOT, weight_name))
print('last-model saved')

100%|██████████| 38/38 [00:04<00:00,  8.50it/s]


best-model saved (epoch: 1)
epoch: 1, loss_train: 1.2316, accuracy_train: 0.3259, loss_valid: 1.2158, accuracy_valid: 0.3651, 76.9760sec


100%|██████████| 38/38 [00:04<00:00,  8.52it/s]


best-model saved (epoch: 2)
epoch: 2, loss_train: 0.8994, accuracy_train: 0.6741, loss_valid: 0.9217, accuracy_valid: 0.6711, 76.6664sec


100%|██████████| 38/38 [00:04<00:00,  8.52it/s]


...

epoch: 8, loss_train: 0.5022, accuracy_train: 0.8144, loss_valid: 0.7841, accuracy_valid: 0.7138, 74.7547sec


100%|██████████| 38/38 [00:04<00:00,  8.50it/s]


best-model saved (epoch: 9)
epoch: 9, loss_train: 0.4579, accuracy_train: 0.8432, loss_valid: 0.7589, accuracy_valid: 0.7303, 76.4565sec


100%|██████████| 38/38 [00:04<00:00,  8.54it/s]


epoch: 10, loss_train: 0.4234, accuracy_train: 0.8490, loss_valid: 0.7745, accuracy_valid: 0.7237, 74.9486sec
last-model saved


In [23]:
# 正解率の算出
def predict(model, dataset, device):
  # Dataloaderの作成
  loader = DataLoader(dataset, batch_size=8, shuffle=False)

  model.eval()
  total = 0
  preds = []
  with torch.no_grad():
    for data in tqdm(loader):
      # デバイスの指定
      ids = data['ids'].to(device)
      mask = data['mask'].to(device)
      labels = data['labels'].to(device)

      # 順伝播 + 予測値の取得 + 正解数のカウント
      outputs = model.forward(ids, mask)
      pred = torch.argmax(outputs, dim=-1).cpu().numpy()
      preds.append(pred)

  return preds
dataset_test = CreateDataset(test['description'], [0]*len(test['description']), tokenizer, max_len)
model = torch.load(os.path.join(ROOT, 'v3_best.pth'))
preds = predict(model, dataset_test, device)

100%|██████████| 190/190 [00:22<00:00,  8.52it/s]


In [25]:
flatten = [p+1 for arr in preds for p in arr]
df_submit = pd.DataFrame([[id, pred] for id, pred in zip(test.id, flatten)])
df_submit.to_csv(os.path.join(ROOT, 'v3_submit.csv'), index=None, header=None)

In [39]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
for dataset_, y_ in zip([dataset_train, dataset_val], [y_train, y_val]):
  preds_ = predict(model, dataset_, device)
  preds_ = [p+1 for arr in preds_ for p in arr]
  labels = [np.argmax(y)+1 for y in y_]
  print('\n', confusion_matrix(preds_, labels))
  print("Classification Report: ")
  print(classification_report(labels, preds_))

100%|██████████| 152/152 [00:17<00:00,  8.45it/s]



 [[326  38  10  50]
 [  2  14   3   0]
 [ 11  19 350  12]
 [ 35   1   9 332]]
Classification Report: 
              precision    recall  f1-score   support

           1       0.77      0.87      0.82       374
           2       0.74      0.19      0.31        72
           3       0.89      0.94      0.92       372
           4       0.88      0.84      0.86       394

    accuracy                           0.84      1212
   macro avg       0.82      0.71      0.73      1212
weighted avg       0.84      0.84      0.83      1212



100%|██████████| 38/38 [00:04<00:00,  8.53it/s]


 [[65 12  3 20]
 [ 0  0  0  0]
 [ 9  4 71  5]
 [20  0  9 86]]
Classification Report: 
              precision    recall  f1-score   support

           1       0.65      0.69      0.67        94
           2       0.00      0.00      0.00        16
           3       0.80      0.86      0.83        83
           4       0.75      0.77      0.76       111

    accuracy                           0.73       304
   macro avg       0.55      0.58      0.56       304
weighted avg       0.69      0.73      0.71       304




/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
